### **Latin America Climate and Air Quality ETL**

#### **Importing Location Data**

In [33]:
import openmeteo_requests
import requests_cache
import pandas as pd
from retry_requests import retry

In [34]:
file_path = './countries.csv'
country_dataframe = pd.read_csv(file_path)

In [35]:
country_list = country_dataframe["country"].tolist()
city_list = country_dataframe["city"].tolist()
latitude_list = country_dataframe["latitude"].tolist()
longitude_list = country_dataframe["longitude"].tolist()

print(city_list)
print(country_list)

[' Buenos Aires', ' Sucre', ' Brasilia', ' Santiago', ' Bogota', ' San Jose', ' Havana', ' Santo Domingo', ' Quito', ' San Salvador', ' Guatemala City', ' Port-au-Prince', ' Tegucigalpa', ' Mexico City', ' Managua', ' Panama City', ' Asuncion', ' Lima', ' Montevideo', ' Caracas']
['Argentina', 'Bolivia', 'Brazil', 'Chile', 'Colombia', 'Costa Rica', 'Cuba', 'Dominican Republic', 'Ecuador', 'El Salvador', 'Guatemala', 'Haiti', 'Honduras', 'Mexico', 'Nicaragua', 'Panama', 'Paraguay', 'Peru', 'Uruguay', 'Venezuela']


#### **Setting up Open-Meteo Connection**

In [36]:
# Configurando o clinete da API do Open-Meteo com cache e tente novamente em caso de erro
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

#### **Request to Open-Meteo Weather API**

In [37]:
climate_url = "https://archive-api.open-meteo.com/v1/archive"
climate_params = {
	"latitude": latitude_list,
	"longitude": longitude_list,
	"start_date": "2020-01-01",
	"end_date": "2024-04-20",
	"daily": ["temperature_2m_max", "temperature_2m_min", "temperature_2m_mean", "apparent_temperature_max", "apparent_temperature_min", "apparent_temperature_mean", "precipitation_sum", "wind_speed_10m_max", "et0_fao_evapotranspiration"]
}

climate_responses = openmeteo.weather_api(climate_url, params=climate_params)

#### **Generating Climate CSV**

In [38]:
climate_dataframe = pd.DataFrame()

for i in range(len(latitude_list)):
    climate_response = climate_responses[i]
    """ print(f"Coordinates {climate_response.Latitude()}°N {climate_response.Longitude()}°E")
    print(f"Elevation {climate_response.Elevation()} m asl")
    print(f"Timezone {climate_response.Timezone()} {climate_response.TimezoneAbbreviation()}")
    print(f"Timezone difference to GMT+0 {climate_response.UtcOffsetSeconds()} s") """

    # Process daily data. The order of variables needs to be the same as requested.
    daily = climate_response.Daily()
    daily_temperature_2m_max = daily.Variables(0).ValuesAsNumpy()
    daily_temperature_2m_min = daily.Variables(1).ValuesAsNumpy()
    daily_temperature_2m_mean = daily.Variables(2).ValuesAsNumpy()
    daily_apparent_temperature_max = daily.Variables(3).ValuesAsNumpy()
    daily_apparent_temperature_min = daily.Variables(4).ValuesAsNumpy()
    daily_apparent_temperature_mean = daily.Variables(5).ValuesAsNumpy()
    daily_precipitation_sum = daily.Variables(6).ValuesAsNumpy()
    daily_wind_speed_10m_max = daily.Variables(7).ValuesAsNumpy()
    daily_et0_fao_evapotranspiration = daily.Variables(8).ValuesAsNumpy()

    daily_data = { "country": country_list[i], 
                "city": city_list[i], 
                "date": pd.date_range(
    	        start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
    	        end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
    	        freq = pd.Timedelta(seconds = daily.Interval()),
    	        inclusive = "left"
    )}
    daily_data["latitude"] = latitude_list[i]
    daily_data["longitude"] = longitude_list[i]
    daily_data["temperature_2m_max"] = daily_temperature_2m_max
    daily_data["temperature_2m_min"] = daily_temperature_2m_min
    daily_data["temperature_2m_mean"] = daily_temperature_2m_mean
    daily_data["apparent_temperature_max"] = daily_apparent_temperature_max
    daily_data["apparent_temperature_min"] = daily_apparent_temperature_min
    daily_data["apparent_temperature_mean"] = daily_apparent_temperature_mean
    daily_data["precipitation_sum"] = daily_precipitation_sum
    daily_data["wind_speed_10m_max"] = daily_wind_speed_10m_max
    daily_data["et0_fao_evapotranspiration"] = daily_et0_fao_evapotranspiration

    daily_climate_dataframe = pd.DataFrame(data = daily_data)
    frames = [climate_dataframe, daily_climate_dataframe]
    climate_dataframe = pd.concat(frames)

climate_csv = climate_dataframe.to_csv('./LA_daily_climate.csv', index=False)
climate_dataframe

,country,city,date,latitude,longitude,temperature_2m_max,temperature_2m_min,temperature_2m_mean,apparent_temperature_max,apparent_temperature_min,apparent_temperature_mean,precipitation_sum,wind_speed_10m_max,et0_fao_evapotranspiration
0,Argentina,Buenos Aires,2020-01-01 00:00:00+00:00,-34.6037,-58.3816,24.767500,19.267500,21.973753,25.061949,18.268499,22.049086,0.6,24.336639,4.160744
1,Argentina,Buenos Aires,2020-01-02 00:00:00+00:00,-34.6037,-58.3816,24.217501,18.317501,21.259167,24.995867,17.999479,20.943785,1.8,27.609911,4.593965
2,Argentina,Buenos Aires,2020-01-03 00:00:00+00:00,-34.6037,-58.3816,25.117500,20.517500,22.373751,27.362978,20.149343,22.910418,0.1,24.203140,5.876242
3,Argentina,Buenos Aires,2020-01-04 00:00:00+00:00,-34.6037,-58.3816,25.617500,20.917501,23.177916,27.975208,21.116510,24.117735,0.0,20.833395,5.913107
4,Argentina,Buenos Aires,2020-01-05 00:00:00+00:00,-34.6037,-58.3816,27.167501,20.217501,24.225832,29.682999,22.054735,25.340597,0.0,18.486622,5.996509
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1567,Venezuela,Caracas,2024-04-16 00:00:00+00:00,10.4806,-66.9036,30.464001,19.014000,24.030664,34.626366,20.114017,27.005964,1.9,12.261158,5.001820
1568,Venezuela,Caracas,2024-04-17 00:00:00+00:00,10.4806,-66.9036,27.864000,20.764000,23.861917,32.559837,22.816622,27.147459,6.6,9.531505,3.589120
1569,Venezuela,Caracas,2024-04-18 00:00:00+00:00,10.4806,-66.9036,27.264000,20.914000,23.261915,31.179459,23.071089,26.784079,4.3,7.920000,2.683167
1570,Venezuela,Caracas,2024-04-19 00:00:00+00:00,10.4806,-66.9036,26.313999,21.114000,23.245253,31.100521,24.506313,27.202951,10.2,6.130579,3.454755


#### **Request to Open-Meteo Air Quality API**

In [39]:
air_quality_url = "https://air-quality-api.open-meteo.com/v1/air-quality"
air_quality_params = {
	"latitude": latitude_list,
	"longitude": longitude_list,
	"hourly": ["pm10", "pm2_5", "carbon_monoxide", "nitrogen_dioxide", "sulphur_dioxide", "ozone"],
	"start_date": "2022-07-29",
	"end_date": "2024-04-20",
	"domains": "cams_global"
}
air_quality_responses = openmeteo.weather_api(air_quality_url, params=air_quality_params)

#### **Generating Air Quality CSV**

In [40]:
air_quality_dataframe = pd.DataFrame()

for i in range(len(latitude_list)):
    air_quality_response = air_quality_responses[i]
    """ print(f"Coordinates {air_quality_response.Latitude()}°N {air_quality_response.Longitude()}°E")
    print(f"Elevation {air_quality_response.Elevation()} m asl")
    print(f"Timezone {air_quality_response.Timezone()} {air_quality_response.TimezoneAbbreviation()}")
    print(f"Timezone difference to GMT+0 {air_quality_response.UtcOffsetSeconds()} s") """

    # Process hourly data. The order of variables needs to be the same as requested.
    hourly = air_quality_response.Hourly()
    hourly_pm10 = hourly.Variables(0).ValuesAsNumpy()
    hourly_pm2_5 = hourly.Variables(1).ValuesAsNumpy()
    hourly_carbon_monoxide = hourly.Variables(2).ValuesAsNumpy()
    hourly_nitrogen_dioxide = hourly.Variables(3).ValuesAsNumpy()
    hourly_sulphur_dioxide = hourly.Variables(4).ValuesAsNumpy()
    hourly_ozone = hourly.Variables(5).ValuesAsNumpy()

    hourly_data = {
                   "date": pd.date_range(
    	            start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
    	            end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
    	            freq = pd.Timedelta(seconds = hourly.Interval()),
    	            inclusive = "left"
    )}
    hourly_data["latitude"] = latitude_list[i]
    hourly_data["longitude"] = longitude_list[i]
    hourly_data["pm10"] = hourly_pm10
    hourly_data["pm2_5"] = hourly_pm2_5
    hourly_data["carbon_monoxide"] = hourly_carbon_monoxide
    hourly_data["nitrogen_dioxide"] = hourly_nitrogen_dioxide
    hourly_data["sulphur_dioxide"] = hourly_sulphur_dioxide
    hourly_data["ozone"] = hourly_ozone

    air_quality_hourly_dataframe = pd.DataFrame(data = hourly_data)
    frames = [air_quality_dataframe, air_quality_hourly_dataframe]
    air_quality_dataframe = pd.concat(frames)

In [41]:
air_quality_dataframe['date'] = pd.to_datetime(air_quality_dataframe['date'])

time = '00:00:00+00:00'
mask = (air_quality_dataframe['date'].dt.time == pd.Timestamp(time).time())
filtered_air_quality_df = air_quality_dataframe.loc[mask]

air_quality_csv = filtered_air_quality_df.to_csv('./LA_daily_air_quality.csv', index=False)
filtered_air_quality_df

,date,latitude,longitude,pm10,pm2_5,carbon_monoxide,nitrogen_dioxide,sulphur_dioxide,ozone
0,2022-07-29 00:00:00+00:00,-34.6037,-58.3816,NaN,NaN,NaN,NaN,NaN,NaN
24,2022-07-30 00:00:00+00:00,-34.6037,-58.3816,NaN,NaN,NaN,NaN,NaN,NaN
48,2022-07-31 00:00:00+00:00,-34.6037,-58.3816,NaN,NaN,NaN,NaN,NaN,NaN
72,2022-08-01 00:00:00+00:00,-34.6037,-58.3816,NaN,NaN,NaN,NaN,NaN,NaN
96,2022-08-02 00:00:00+00:00,-34.6037,-58.3816,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
15048,2024-04-16 00:00:00+00:00,10.4806,-66.9036,9.300000,6.100000,232.0,7.3,2.1,41.0
15072,2024-04-17 00:00:00+00:00,10.4806,-66.9036,28.500000,19.600000,510.0,18.5,3.4,36.0
15096,2024-04-18 00:00:00+00:00,10.4806,-66.9036,23.100000,16.200001,446.0,14.2,3.2,58.0
15120,2024-04-19 00:00:00+00:00,10.4806,-66.9036,40.700001,29.100000,775.0,26.5,5.6,49.0
